In [2]:
import os
import shutil

In [3]:
!ls

Data  drive  sample_data


In [4]:
source='drive/My Drive/Dataset/Dataset.zip'
destination='Data'
shutil.copy2(source,destination)

'Data/Dataset.zip'

In [5]:
!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/
   creating: Dataset/test/
   creating: Dataset/test/covid/
  inflating: Dataset/test/covid/gr1_lrg-b.jpg  
  inflating: Dataset/test/covid/kjr-21-e25-g001-l-a.jpg  
  inflating: Dataset/test/covid/lancet-case2a.jpg  
  inflating: Dataset/test/covid/lancet-case2b.jpg  
  inflating: Dataset/test/covid/nCoV-radiol.2020200269.fig1-day7.jpeg  
  inflating: Dataset/test/covid/nejmc2001573_f1a.jpeg  
  inflating: Dataset/test/covid/nejmc2001573_f1b.jpeg  
  inflating: Dataset/test/covid/nejmoa2001191_f1-PA.jpeg  
  inflating: Dataset/test/covid/nejmoa2001191_f3-PA.jpeg  
  inflating: Dataset/test/covid/nejmoa2001191_f4.jpeg  
  inflating: Dataset/test/covid/nejmoa2001191_f5-PA.jpeg  
  inflating: Dataset/test/covid/paving.jpg  
  inflating: Dataset/test/covid/radiol.2020200274.fig3d.png  
  inflating: Dataset/test/covid/radiol.2020200490.fig3.jpeg  
  inflating: Dataset/test/covid/ryct.2020200028.fig1a.jpeg  
  inflating: Dataset/test/covid/ryct.20

In [6]:
tarin_path='Dataset/train'
val_path='Dataset/test'

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [9]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(32,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 54, 54, 64)       

In [13]:
tarin_datagen=image.ImageDataGenerator(
    rescale=1./255,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
)

test_datset=image.ImageDataGenerator(rescale=1./255)


In [14]:
tarin_gen=tarin_datagen.flow_from_directory(
    'Dataset/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 314 images belonging to 2 classes.


In [15]:
tarin_gen.class_indices

{'covid': 0, 'normal': 1}

In [17]:
validation_generator=test_datset.flow_from_directory(
    'Dataset/test',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 66 images belonging to 2 classes.


In [18]:
hist=model.fit_generator(
    tarin_gen,
    steps_per_epoch=8,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=2
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
8/8 [==============================] - 9s 1s/step - loss: 1.4564 - accuracy: 0.5320 - val_loss: 0.6889 - val_accuracy: 0.6562
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6862 - accuracy: 0.5640 - val_loss: 0.6842 - val_accuracy: 0.6875
Epoch 3/10
8/8 [==============================] - 8s 1s/step - loss: 0.6583 - accuracy: 0.5898 - val_loss: 0.5689 - val_accuracy: 0.8906
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.5500 - accuracy: 0.7200 - val_loss: 0.3608 - val_accuracy: 0.9375
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.4021 - accuracy: 0.8480 - val_loss: 0.1883 - val_accuracy: 0.9531
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.3632 - accuracy: 0.8281 - val_loss: 0.2197 - val_accuracy: 0.9688
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.4189 - accuracy: 0.8560 - val_l

In [21]:
model.save('covidpred.h5')

In [24]:
(model.predict(validation_generator))

array([[0.98949426],
       [0.99609596],
       [0.99852604],
       [0.9359362 ],
       [0.96441597],
       [0.9867278 ],
       [0.14425589],
       [0.9909429 ],
       [0.9906495 ],
       [0.61418784],
       [0.9925477 ],
       [0.01253523],
       [0.96308124],
       [0.993594  ],
       [0.12137789],
       [0.08572339],
       [0.9524323 ],
       [0.9898297 ],
       [0.01449854],
       [0.14296663],
       [0.7904972 ],
       [0.9879247 ],
       [0.9874604 ],
       [0.94043505],
       [0.34706017],
       [0.5514326 ],
       [0.9920449 ],
       [0.99581134],
       [0.9964921 ],
       [0.96662056],
       [0.9877921 ],
       [0.97209394],
       [0.98900336],
       [0.06305917],
       [0.08240833],
       [0.988089  ],
       [0.9634641 ],
       [0.992477  ],
       [0.99241626],
       [0.20834382],
       [0.06725127],
       [0.9758941 ],
       [0.01917109],
       [0.07332204],
       [0.03275498],
       [0.9984908 ],
       [0.9673813 ],
       [0.024

In [25]:
model.evaluate_generator(validation_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.12527459859848022, 0.9696969985961914]

In [27]:
model.evaluate_generator(tarin_gen)

[0.17058661580085754, 0.9363057613372803]